In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sakshi1234kumar","key":"364e599645bf44e8a0b1c01badf7dc73"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle


In [3]:
import zipfile

zip_path = "/content/drive/MyDrive/sign_lang/asl-alphabet.zip"
extract_path = "/content/drive/MyDrive/sign_lang/asl_alphabet_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


KeyboardInterrupt: 

In [ ]:
data_dir = '/content/drive/MyDrive/sign_lang/asl_alphabet_data/asl_alphabet_train/sub'


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,         # Convert pixel values from 0–255 to 0–1
    validation_split=0.2    # 20% of data is used for validation
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),       # Resize all images to 64x64
    color_mode='grayscale',     # Images are in grayscale (1 channel)
    batch_size=32,              # Process 32 images at a time
    class_mode='categorical',   # Output will be one-hot encoded
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 2160 images belonging to 27 classes.
Found 539 images belonging to 27 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),  # 1st Conv layer
    MaxPooling2D(2, 2),                                              # Reduces size

    Conv2D(64, (3, 3), activation='relu'),                           # 2nd Conv layer
    MaxPooling2D(2, 2),                                              # Again downsampling

    Flatten(),                      # Converts 2D features to 1D vector
    Dense(256, activation='relu'),  # Fully connected layer
    Dropout(0.3),                   # Prevents overfitting
    Dense(train_data.num_classes, activation='softmax')  # Output layer (29 classes)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 27)             │         6,939 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,237,275 (12.35 MB)

 Trainable params: 3,237,275 (12.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 819s 12s/step - accuracy: 0.1973 - loss: 2.9428 - val_accuracy: 0.5529 - val_loss: 1.5180
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.8272 - loss: 0.6202 - val_accuracy: 0.8145 - val_loss: 0.7613
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 115ms/step - accuracy: 0.9715 - loss: 0.1541 - val_accuracy: 0.8237 - val_loss: 0.7535
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.9779 - loss: 0.0766 - val_accuracy: 0.8738 - val_loss: 0.6256
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 120ms/step - accuracy: 0.9912 - loss: 0.0432 - val_accuracy: 0.8887 - val_loss: 0.6447
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - accuracy: 0.9935 - loss: 0.0278 - val_accuracy: 0.8868 - val_loss: 0.6756
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 8s 121ms/step - accuracy: 0.9925 - loss: 0.0270 - val_accuracy: 0.8757 - val_loss: 0.6803
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 9s 129ms/step - accuracy: 0.9924 - loss: 0.0298 - val_accuracy: 0.